In [1]:
surname = "Кандрахин" #Ваша фамилия

alp = 'абвгдеёжзийклмнопрстуфхцчшщъыьэюя'
w = [1, 42, 21, 21, 34,  6, 44, 26, 18, 44, 38, 26, 14, 43,  4, 49, 45,
        7, 42, 29,  4,  9, 36, 34, 31, 29,  5, 30,  4, 19, 28, 25, 33]

d = dict(zip(alp, w))
variant =  sum([d[el] for el in surname.lower()]) % 40 + 1

print("Задача № 1: ", variant % 3 + 1)
print("Задача № 2: ", variant % 2 + 1 )

Задача № 1:  3
Задача № 2:  1


In [2]:
genres = ["Animation", "Romance", "Documentary"]

ubuntu@linux:~/BigData/hadoop/bin$ ./hdfs dfs -copyFromLocal /home/ubuntu/Desktop/ml-latest-small/movies.csv /HW4_movies

ubuntu@linux:~/BigData/hadoop/bin$ ./hdfs dfs -copyFromLocal /home/ubuntu/Desktop/ml-latest-small/ratings.csv /HW4_ratings

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode, split, col, count, desc,  collect_list, slice


In [4]:
spark = SparkSession.builder.appName("MovieAnalyzerTemp").getOrCreate()

# Задание 1. Анализ датасета

## Выведите данные, сопоставляющие жанры и количество фильмов


In [5]:
movies_df = spark.read.csv("/HW4_movies", header=True)
ratings_df = spark.read.csv("/HW4_ratings", header=True)

In [6]:
# Парсим жанры и разносим построчно, создавая новые экземпляры
movies_df = movies_df.withColumn("genre", explode(split(col("genres"), "\|")))

In [7]:
# Группируем по жанру и считаем
genre_counts = movies_df.groupBy("genre").count()
genre_counts.show()

+------------------+-----+
|             genre|count|
+------------------+-----+
|             Crime| 1199|
|           Romance| 1596|
|          Thriller| 1894|
|         Adventure| 1263|
|             Drama| 4361|
|               War|  382|
|       Documentary|  440|
|           Fantasy|  779|
|           Mystery|  573|
|           Musical|  334|
|         Animation|  611|
|         Film-Noir|   87|
|(no genres listed)|   34|
|              IMAX|  158|
|            Horror|  978|
|           Western|  167|
|            Comedy| 3756|
|          Children|  664|
|            Action| 1828|
|            Sci-Fi|  980|
+------------------+-----+



## Выведите первые 10 фильмов с наибольшим количеством рейтингов для каждого жанра в соответствии с вариантом

In [8]:
top_rated_movies = (movies_df
                    .join(ratings_df, movies_df.movieId == ratings_df.movieId)
                    .groupBy("genre", "title")
                    .count()
                    .filter(col("genre").isin(genres))
                    .withColumnRenamed("count", "rating_count")
                    .orderBy("genre", desc("rating_count"))
                    .groupBy("genre")
                    .agg(collect_list("title").alias("top_movies"))
                    .select("genre", slice(col("top_movies"), 1, 10).alias("top_10_movies")))

In [9]:
for row in top_rated_movies.rdd.collect():
    genre = row['genre']
    top_10_movies = row['top_10_movies']
    
    print(f"\nЖанр: {genre}\n")
    for movie in top_10_movies:
        print(f"{movie}")


Жанр: Romance

Forrest Gump (1994)
American Beauty (1999)
True Lies (1994)
Speed (1994)
Shrek (2001)
Beauty and the Beast (1991)
Groundhog Day (1993)
Princess Bride, The (1987)
Good Will Hunting (1997)
Titanic (1997)

Жанр: Documentary

Bowling for Columbine (2002)
Super Size Me (2004)
Fahrenheit 9/11 (2004)
Roger & Me (1989)
Hoop Dreams (1994)
March of the Penguins (Marche de l'empereur, La) (2005)
Crumb (1994)
Jackass: The Movie (2002)
Sicko (2007)
Spellbound (2002)

Жанр: Animation

Toy Story (1995)
Aladdin (1992)
Lion King, The (1994)
Shrek (2001)
Beauty and the Beast (1991)
Finding Nemo (2003)
Monsters, Inc. (2001)
Incredibles, The (2004)
Up (2009)
WALL·E (2008)


## Выведите первые 10 фильмов с наименьшим количеством рейтингов (но больше 10) для каждого жанра в соответствии с вариантом

In [10]:
from pyspark.sql.window import Window
from pyspark.sql.functions import rank, row_number, avg


In [11]:
# посчитаем количество рейтинга для каждого фильма в каждом жанре
movie_ratings_count = (movies_df
                       .join(ratings_df, movies_df.movieId == ratings_df.movieId)
                       .filter(col("genre").isin(genres))
                       .groupBy("genre", "title")
                       .count()
                       .filter(col("count") > 10)
                       .withColumnRenamed("count", "rating_count"))

In [12]:
#movie_ratings_count.show()

In [13]:
# Отсортируем по количеству рейтингов
windowSpec = Window.partitionBy("genre").orderBy(col("rating_count").asc())
movies_least_ratings = (movie_ratings_count
                        .withColumn("row_number", row_number().over(windowSpec))
                        .filter(col("row_number") <= 10)
                        .select("genre", "title", "rating_count"))


In [14]:
movies_least_ratings.show(movies_least_ratings.count(), truncate=True)

+-----------+--------------------+------------+
|      genre|               title|rating_count|
+-----------+--------------------+------------+
|    Romance|Across the Univer...|          11|
|    Romance|All Dogs Go to He...|          11|
|    Romance|From Here to Eter...|          11|
|    Romance| Medicine Man (1992)|          11|
|    Romance|Great Expectation...|          11|
|    Romance|Autumn in New Yor...|          11|
|    Romance|   Persuasion (1995)|          11|
|    Romance|This Means War (2...|          11|
|    Romance|Science of Sleep,...|          11|
|    Romance|When a Man Loves ...|          11|
|Documentary|Koyaanisqatsi (a....|          11|
|Documentary|Thin Blue Line, T...|          11|
|Documentary|King of Kong, The...|          12|
|Documentary|   Inside Job (2010)|          12|
|Documentary|   Spellbound (2002)|          13|
|Documentary|Exit Through the ...|          13|
|Documentary|Inconvenient Trut...|          13|
|Documentary|Fog of War: Eleve...|      

## Выведите первые 10 фильмов с наибольшим средним рейтингом при количестве рейтингов больше 10 для каждого жанра в соответствии с вариантом

In [15]:
movies_avg_rating = (movies_df
                     .join(ratings_df, movies_df.movieId == ratings_df.movieId)
                     .filter(col("genre").isin(genres))
                     .groupBy("genre", "title")
                     .agg(avg("rating").alias("average_rating"), count("rating").alias("rating_count"))
                     .filter(col("rating_count") > 10))

In [16]:
windowSpec = Window.partitionBy("genre").orderBy(col("average_rating").desc())


In [17]:
# Применяем оконную функцию row_number и выбираем первые 10 фильмов для каждого жанра
top_movies_by_avg_rating = (movies_avg_rating
                            .withColumn("row_number", row_number().over(windowSpec))
                            .filter(col("row_number") <= 10)
                            .select("genre", "title", "average_rating"))


In [18]:
top_movies_by_avg_rating.show(movies_least_ratings.count(), truncate=False)

+-----------+--------------------------------------------------------------------------+------------------+
|genre      |title                                                                     |average_rating    |
+-----------+--------------------------------------------------------------------------+------------------+
|Romance    |His Girl Friday (1940)                                                    |4.392857142857143 |
|Romance    |Sunset Blvd. (a.k.a. Sunset Boulevard) (1950)                             |4.333333333333333 |
|Romance    |It Happened One Night (1934)                                              |4.321428571428571 |
|Romance    |Philadelphia Story, The (1940)                                            |4.310344827586207 |
|Romance    |Harold and Maude (1971)                                                   |4.288461538461538 |
|Romance    |Notorious (1946)                                                          |4.25              |
|Romance    |Casablanca (194

## Выведите первые 10 фильмов с наименьшим средним рейтингом при количестве рейтингов больше 10 для каждого жанра в соответствии с вариантом

In [19]:
windowSpec = Window.partitionBy("genre").orderBy(col("average_rating").asc())

In [20]:
low_movies_by_avg_rating = (movies_avg_rating
                               .withColumn("row_number", row_number().over(windowSpec))
                               .filter(col("row_number") <= 10)
                               .select("genre", "title", "average_rating"))


In [21]:
low_movies_by_avg_rating.show(movies_least_ratings.count(), truncate=False)

+-----------+----------------------------------------------------------------+------------------+
|genre      |title                                                           |average_rating    |
+-----------+----------------------------------------------------------------+------------------+
|Romance    |Speed 2: Cruise Control (1997)                                  |1.605263157894737 |
|Romance    |Grease 2 (1982)                                                 |2.0789473684210527|
|Romance    |Bewitched (2005)                                                |2.269230769230769 |
|Romance    |Next Karate Kid, The (1994)                                     |2.3666666666666667|
|Romance    |Joe Dirt (2001)                                                 |2.380952380952381 |
|Romance    |Look Who's Talking (1989)                                       |2.388888888888889 |
|Romance    |Autumn in New York (2000)                                       |2.409090909090909 |
|Romance    |Twiligh

# Задание 2. Коллаборативная фильтрация
### Вариант 1. По схожести пользователей

In [22]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql.functions import col, avg, lit, udf, corr
from pyspark.sql import functions as F


In [23]:
ratings_df = ratings_df.withColumn("userId", ratings_df["userId"].cast("integer"))
ratings_df = ratings_df.withColumn("movieId", ratings_df["movieId"].cast("integer"))
ratings_df = ratings_df.withColumn("rating", ratings_df["rating"].cast("float"))

In [24]:
(train_init, test) = ratings_df.randomSplit([0.8, 0.2], seed=3)

In [25]:
average_rating = train_init.select(avg("rating")).first()[0]

In [26]:
test = test.withColumn("avg_rating", F.lit(average_rating))
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="avg_rating")
rmse = evaluator.evaluate(test)
print("Root-mean-square error = " + str(rmse))

Root-mean-square error = 1.0501373838186079


таблица схожести пользователей, pirson correlation

In [27]:
user_similarity = train_init.alias("r1").join(train_init.alias("r2"), ["movieId"]) \
    .filter(col("r1.userId") != col("r2.userId")) \
    .groupBy(col("r1.userId").alias("user1Id"), col("r2.userId").alias("user2Id")) \
    .agg(corr("r1.rating", "r2.rating").alias("similarity")) \
    .na.fill(0)

In [28]:
predicted_ratings = test.alias("t") \
    .join(user_similarity.alias("u"), col("t.userId") == col("u.user1Id")) \
    .join(train_init.alias("r"), (col("u.user2Id") == col("r.userId")) & (col("r.movieId") == col("t.movieId"))) \
    .withColumn("weighted_rating", col("r.rating") * col("u.similarity")) \
    .groupBy("t.userId", "t.movieId") \
    .agg(F.sum("weighted_rating").alias("sum_weighted_rating"), F.sum("u.similarity").alias("sum_similarity")) \
    .withColumn("predicted_rating", col("sum_weighted_rating") / col("sum_similarity"))


In [29]:
predicted_ratings = predicted_ratings.na.drop(subset=["predicted_rating"])

In [30]:
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="predicted_rating")
rmse = evaluator.evaluate(predicted_ratings.join(test, ["userId", "movieId"]))
print("rmse = " + str(rmse))

rmse = 129.75472341732237


# Задание 3. Факторизация матрицы


In [32]:
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

In [33]:
als = ALS(userCol="userId", itemCol="movieId", ratingCol="rating", coldStartStrategy="drop", nonnegative=True)

In [34]:
paramGrid = ParamGridBuilder() \
    .addGrid(als.rank, [5, 10, 15]) \
    .addGrid(als.regParam, [0.001, 0.01, 0.1, 1, 10]) \
    .build()

In [35]:
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")

In [36]:
crossval = CrossValidator(estimator=als,
                          estimatorParamMaps=paramGrid,
                          evaluator=evaluator,
                          numFolds=4) 

In [37]:
cvModel = crossval.fit(train_init)

In [38]:
predictions = cvModel.transform(test)

In [39]:
rmse = evaluator.evaluate(predictions)

In [40]:
print("rmse = " + str(rmse))


rmse = 0.8879947697856218


ALS оказался гораздо эффективнее чем использование корреляции Пирсона и регрессии

In [41]:
best_model = cvModel.bestModel
best_rank = best_model.rank
best_reg_param = best_model._java_obj.parent().getRegParam()

print("Лучший rank:", best_rank)
print("Лучший regParam:", best_reg_param)

Лучший rank: 10
Лучший regParam: 0.1


In [ ]:
# best_rank = 10
# best_reg_param = 0.1
# final_als = ALS(userCol="userId", itemCol="movieId", ratingCol="rating", coldStartStrategy="drop", nonnegative=True,
#                 rank=best_rank, regParam=best_reg_param)
# final_model = final_als.fit(train_init)
# final_predictions = final_model.transform(test)
# final_rmse = evaluator.evaluate(final_predictions)